<a href="https://colab.research.google.com/github/mancinimassimiliano/DeepLearningLab/blob/master/myFirstNN_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision

DEVICE='cuda:0'
LR=0.1
WEIGHT_DECAY = 0.000001 
MOMENTUM = 0.9
BATCH_SIZE = 256
TEST_BATCH_SIZE = 500
EPOCHS = 20

INPUT_DIM = 784
HIDDEN_DIM = 100
OUTPUT_DIM = 10


class MyFirstNetwork(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(MyFirstNetwork, self).__init__()
    self.input_to_hidden = nn.Linear(input_dim, hidden_dim)
    self.activation = nn.ReLU()
    self.hidden_to_output = nn.Linear(hidden_dim, output_dim)
    
    self.input_to_hidden.weight.data.normal_(0,0.1)
    self.input_to_hidden.bias.data.fill_(0.0)
    self.hidden_to_output.weight.data.normal_(0,0.1)
    self.hidden_to_output.bias.data.fill_(0.0)

  def forward(self, x):
    x = x.view(x.shape[0],-1)
    x = self.input_to_hidden(x)
    x = self.activation(x)
    x = self.hidden_to_output(x)
    return x
  

def get_data(): 
  transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

  full_training_data = torchvision.datasets.MNIST('.', train=True, download=True, transform=transform) 
  test_data = torchvision.datasets.MNIST('.', train=False, download=True, transform=transform)
  


  num_samples = len(full_training_data)
  training_samples = int(num_samples*0.5+1)
  validation_samples = num_samples - training_samples

  training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples, validation_samples])

  train_loader = torch.utils.data.DataLoader(training_data, batch_size = BATCH_SIZE, shuffle=True)
  val_loader = torch.utils.data.DataLoader(validation_data, batch_size = TEST_BATCH_SIZE, shuffle=False)
  test_loader = torch.utils.data.DataLoader(test_data, batch_size = TEST_BATCH_SIZE, shuffle=False)
  
  return train_loader, val_loader, test_loader


def get_optimizer(net, lr, wd, momentum):
  optimizer = optim.SGD(net.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
  return optimizer


def get_cost_function():
  return nn.CrossEntropyLoss()



def test(net, data_loader, cost_function):
  net.eval()
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(data_loader):
      # Load data into GPU
      inputs = inputs.to(DEVICE)
      targets = targets.to(DEVICE)
        
      # Forward pass
      outputs = net(inputs)

      # Apply the loss
      loss = cost_function(outputs,targets)

      # Better print something, no?
      samples+=inputs.shape[0]
      cumulative_loss += loss.item()
      _, predicted = outputs.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100




def train(net,data_loader,optimizer,cost_function):
  net.train()
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  for batch_idx, (inputs, targets) in enumerate(data_loader):
    # Load data into GPU
    inputs = inputs.to(DEVICE)
    targets = targets.to(DEVICE)
      
    # Forward pass
    outputs = net(inputs)

    # Apply the loss
    loss = cost_function(outputs,targets)

    # Reset the optimizer
    optimizer.zero_grad()
      
    # Backward pass
    loss.backward()
      
    # Update parameters
    optimizer.step()

    # Better print something, no?
    samples+=inputs.shape[0]
    cumulative_loss += loss.item()
    _, predicted = outputs.max(1)
    cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


def main():
  train_loader, val_loader, test_loader = get_data()
  net = MyFirstNetwork(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM).to(DEVICE)
  optimizer = get_optimizer(net, LR, WEIGHT_DECAY, MOMENTUM)
  cost_function = get_cost_function()

  print('Before training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function)
  val_loss, val_accuracy = test(net, val_loader, cost_function)
  test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

  for e in range(EPOCHS):
    train_loss, train_accuracy = train(net, train_loader, optimizer, cost_function)
    val_loss, val_accuracy = test(net, val_loader, cost_function)
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')

  print('After training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function)
  val_loss, val_accuracy = test(net, val_loader, cost_function)
  test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

main()

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 26975597.39it/s]                            


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 408587.28it/s]
0it [00:00, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 6735366.45it/s]                           
8192it [00:00, 168031.43it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!
Before training:
	 Training loss 0.00954, Training accuracy 11.98
	 Validation loss 0.00486, Validation accuracy 11.86
	 Test loss 0.00489, Test accuracy 11.27
-----------------------------------------------------
Epoch: 1
	 Training loss 0.00195, Training accuracy 84.96
	 Validation loss 0.00056, Validation accuracy 91.82
-----------------------------------------------------
Epoch: 2
	 Training loss 0.00084, Training accuracy 93.63
	 Validation loss 0.00039, Validation accuracy 94.36
-----------------------------------------------------
Epoch: 3
	 Training loss 0.00060, Training accuracy 95.54
	 Validation loss 0.00032, Validation accuracy 95.26
-----------------------------------------------------
Epoch: 4
	 Training loss 0.00046, Training accuracy 96.56
	 Validation loss 0.00030, Validation accuracy 95.54
-----------------------------------------------------
Epoch: 5